# BERT를 처음 사용하기 위한 시각적 가이드 및 분류 문제 해결

- 이번 과제에서는 BERT 모델을 설치해보고 이를 활용하여 모델을 불러오고, 분류 문제를 풀기 위해 모델을 학습해본다.
- 문장 2000개를 DistilBERT를 통해 전처리 (BertEmbedding) 결과를 받아 문장을 긍정 또는 부정 (각각 1 또는 0)으로 분류하는 간단한 Logistic Regression을 구현한다.
- 이번 과제는 아래의 튜토리얼에서 발췌되었고, 과제의 보다 자세한 내용을 위해서 아래 튜토리얼을 참고하면 됩니다.

- http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/ [영어]
- https://chloamme.github.io/2021/12/22/a-visual-guide-to-using-bert-for-the-first-time-korean.html [한국어]

- 이번 과제에서 풀어야 할 내용
- 0. 튜토리얼 실행 [10점]
- 1. 튜토리얼에 이용한 전처리 문장 수를 2000개에서 전체 6915개를 모두 이용하여 이전 Logistic Regression 모델의 결과와 비교한다. [10점]
- 2. DistilBERT를 이용해 전처리된 내용을 Logistic Regression 외에 LSTM, GRU, BertForSequenceClassification (택1) 등을 이용하여 Logistic Regression의 성능과 비교한다. [20점]

## Task: 문장 감정 분류
 
- 우리의 목표는 (우리의 데이터셋과 같은) 문장을 받아서 (긍정적 감정을 가진 문장을 의미하는) 1 또는 (부정적 감정을 가진 문장을 의미하는) 0을 생성하는 것입니다. 다음과 같이 생겼다고 생각할 수 있습니다:
Our goal is to create a model that takes a sentence (just like the ones in our dataset) and produces either 1 (indicating the sentence carries a positive sentiment) or a 0 (indicating the sentence carries a negative sentiment). We can think of it as looking like this:

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

- 모델은 사실 내부적으로 두개의 모델로 구성되어 있습니다.
 
- DistilBERT는 문장을 처리하고, 문장에서 추출한 몇가지 정보를 다음 모델에 전달합니다. DistilBERT는 HuggingFace 팀에서 개발하고 오픈소스로 제공하는 BERT의 작은 버전입니다. 성능은 BERT와 대략 비슷하지만 가볍고 빠른 버전입니다. 다음 모델인 scikit learn의 기본 Logistic Regression 모델은 DistilBERT 처리 결과를 받아 문장을 긍정 또는 부정 (각각 1 또는 0)으로 분류합니다.
 
- 두 모델 간 전달하는 데이터는 768 크기의 벡터입니다. 벡터를 분류하려는 문장에 대한 임베딩으로 생각할 수 있습니다.


<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />

## Dataset
이 예제에서 사용할 데이터셋은 SST2이고, 이 것은 영화 리뷰 문장들과, 각 문장 별 긍정(값이 1) 또는 부정(값이 0)으로 레이블링 되어 있습니다:


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>


## Importing the dataset
데이터 로드

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('sst-train.tsv', delimiter='\t', header=None)

In [2]:
df

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0


본 과제에서는 2000개의 문장만 이용하기로 한다.

In [3]:
batch_1 = df[:2000]
batch_1.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


- Dataframe을 이용해 얼마나 많은 문장이 "긍정적"(값 1)으로 표시되고 얼마나 많은 문장이 "부정적"(값 0)으로 표시되는지 value_counts()라는 함수를 이용해 출력할 수 있다.

In [4]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## Installing the transformers library
- 딥러닝 NLP 모델을 로드할 수 있도록 pytorch와 huggingface transformer 라이브러리를 설치하는 것으로 시작하겠습니다.
- pytorch의 경우 OS에 따라 https://pytorch.org/get-started/locally/ 링크를 참고하여 설치할 수 있도록 한다.
- 이미 라이브러리를 설치 했다면 이 과정은 건너 뛰어도 됩니다.

In [5]:
# CUDA 설치 for Windows
# 학습을 위한 그래픽 카드를 사용하기 위한 CUDA 설치는 아래 링크를 참고
# https://afsdzvcx123.tistory.com/entry/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5-Windows%EC%9C%88%EB%8F%84%EC%9A%B0-CUDA-cuDNN-%EC%84%A4%EC%B9%98%EB%B0%A9%EB%B2%95

# pytorch - https://pytorch.org/get-started/locally/
#pytorch 설치 for Windows - 2.4GB 정도의 되는 라이브러리를 설치함. 인터넷 상태에 따라 10분 이상 걸림
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

# transformers 설치
# !pip install transformers

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## distilBERT 대신 BERT를 사용하고 싶으신가요? 그럼 아래의 주석처리를 해제하세요
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# 현재 'model' 변수는 사전 훈련된 distilBERT 모델을 보유하고 있습니다. 이 모델은 더 작지만 훨씬 빠르고 훨씬 적은 메모리를 필요로 하는 BERT 버전입니다.

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Model #1: Preparing the Dataset
- 문장을 BERT에 전달하기 전에 필요한 형식으로 문장을 넣기 위한 최소한의 처리가 필요합니다. 이를 Tokenization이라고 부릅니다.

### Tokenization
첫 번째 단계는 문장을 토큰화하는 것입니다. BERT가 편한 형식으로 데이터셋에 있는 문장을 word와 sub word로 분해합니다.

In [7]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized

0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
1995    [101, 2205, 20857, 1998, 11865, 16643, 2135, 5...
1996    [101, 2009, 2515, 1050, 1005, 1056, 2147, 2004...
1997    [101, 2023, 2028, 8704, 2005, 1996, 11848, 199...
1998    [101, 1999, 1996, 2171, 1997, 2019, 9382, 1898...
1999    [101, 1996, 3185, 2003, 25757, 2011, 1037, 244...
Name: 0, Length: 2000, dtype: object

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
- 토큰화 후 `tokenized`는 문장의 목록입니다. 각 문장은 토큰 목록으로 표시됩니다. 우리는 BERT가 예제를 한 번에(하나의 배치로) 처리하기를 원합니다. 그렇게 하면 더 빠를 뿐입니다. 이러한 이유로 모든 목록을 동일한 크기로 채워야 입력을 (길이가 다른) 목록 목록이 아닌 하나의 2-d 배열로 나타낼 수 있습니다.

In [8]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded

array([[  101,  1037, 18385, ...,     0,     0,     0],
       [  101,  4593,  2128, ...,     0,     0,     0],
       [  101,  2027,  3653, ...,     0,     0,     0],
       ...,
       [  101,  2023,  2028, ...,     0,     0,     0],
       [  101,  1999,  1996, ...,     0,     0,     0],
       [  101,  1996,  3185, ...,     0,     0,     0]])

- tokenized와 padding을 거친 데이터는 이제 `padded` 변수에 있으며 아래에서 해당 크기를 볼 수 있습니다.

In [9]:
np.array(padded).shape

(2000, 59)

### Masking
- BERT에 `padded`를 직접 보내면 약간 혼란스러울 것입니다. 입력을 처리할 때 추가한 패딩을 무시(마스크)하도록 다른 변수를 만들어야 합니다. 이것이 바로 attention_mask입니다.

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

## Model #1: And Now, Deep Learning!
- 이제 모델과 전처리 된 입력 (tokenized, padded, attention mask) 이 준비되었으므로 모델을 실행해 보겠습니다.

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

- `model()` 함수는 BERT를 통해 문장을 실행합니다. 처리 결과는 `last_hidden_states`로 반환됩니다.


In [11]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

- 출력에서 필요한 부분만 슬라이스해 보겠습니다. 그것은 각 문장의 첫 번째 토큰에 해당하는 출력입니다. BERT가 문장 분류를 수행하는 방식은 모든 문장의 시작 부분에 `[CLS]`(분류용)라는 토큰을 추가하는 것입니다. 해당 토큰에 해당하는 출력은 전체 문장에 대한 임베딩으로 생각할 수 있습니다.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

- 이후에 사용할 로지틱스 회귀 모델의 기능으로 `last_hidden_states`의 정보가 사용되므로 `features` 변수에 저장합니다.

In [12]:
features = last_hidden_states[0][:,0,:].numpy()
features

array([[-0.21593435, -0.14028911,  0.00831076, ..., -0.13694832,
         0.5867005 ,  0.20112693],
       [-0.17262718, -0.14476153,  0.00223438, ..., -0.1744257 ,
         0.21386446,  0.37197465],
       [-0.05063373,  0.07203954, -0.02959727, ..., -0.0714895 ,
         0.7185238 ,  0.2622547 ],
       ...,
       [-0.27829778, -0.24803615,  0.13585788, ..., -0.19039172,
         0.13099575,  0.3497837 ],
       [-0.03667723,  0.10638573, -0.01111037, ..., -0.1120664 ,
         0.41619483,  0.5033802 ],
       [ 0.12402646,  0.01425165,  0.01038418, ..., -0.11606564,
         0.5345917 ,  0.27495325]], dtype=float32)

- 어떤 문장이 긍정적이고 부정적인지를 나타내는 레이블은 이제 `labels` 변수로 이동합니다.

In [13]:
labels = batch_1[1]
labels

0       1
1       0
2       0
3       1
4       1
       ..
1995    0
1996    0
1997    0
1998    0
1999    0
Name: 1, Length: 2000, dtype: int64

## Model #2: Train/Test Split
- 이제 데이터 세트를 훈련 세트와 테스트 세트로 분할하겠습니다(SST2 훈련 세트에서 2,000개의 문장).

In [14]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png" />

In [15]:
### [Bonus] Grid Search for Parameters
# We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

- 이제 LogisticRegression 모델을 훈련합니다. 그리드 검색을 수행하도록 선택한 경우 C 값을 모델 선언에 연결할 수 있습니다(예: `LogisticRegression(C=5.2)`).

In [16]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating Model #2
- 그렇다면 우리 모델이 문장을 분류하는 데 얼마나 잘 작동할까요? 한 가지 방법은 테스트 데이터 세트에 대해 정확도를 확인하는 것입니다.

In [17]:
lr_clf.score(test_features, test_labels)

0.868

.

.

# 전체 문장 이용

**전체 문장을 다 이용하는 batch_2 정의**

In [18]:
import numpy as np
import pandas as pd
df = pd.read_csv('sst-train.tsv', delimiter='\t', header=None)

In [19]:
batch_2 = df
batch_2.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [20]:
batch_2[1].value_counts()

1    3610
0    3310
Name: 1, dtype: int64

**전체 문장에 대해서 토큰화**

In [21]:
tokenized_2 = batch_2[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_2

0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
6915    [101, 9145, 1010, 7570, 18752, 14116, 1998, 28...
6916    [101, 2202, 2729, 2003, 19957, 2864, 2011, 103...
6917    [101, 1996, 5896, 4472, 4121, 1010, 3082, 7832...
6918    [101, 1037, 5667, 2919, 2143, 2007, 5667, 2561...
6919    [101, 1037, 12090, 2135, 2512, 5054, 19570, 23...
Name: 0, Length: 6920, dtype: object

In [22]:
tokenized_2

0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
6915    [101, 9145, 1010, 7570, 18752, 14116, 1998, 28...
6916    [101, 2202, 2729, 2003, 19957, 2864, 2011, 103...
6917    [101, 1996, 5896, 4472, 4121, 1010, 3082, 7832...
6918    [101, 1037, 5667, 2919, 2143, 2007, 5667, 2561...
6919    [101, 1037, 12090, 2135, 2512, 5054, 19570, 23...
Name: 0, Length: 6920, dtype: object

**전체 문장을 다 사용하는 token에 padded_2 추가**

In [23]:
max_len = 0
for i in tokenized_2.values:
    if len(i) > max_len:
        max_len = len(i)

padded_2 = np.array([i + [0]*(max_len-len(i)) for i in tokenized_2.values])
padded_2

array([[  101,  1037, 18385, ...,     0,     0,     0],
       [  101,  4593,  2128, ...,     0,     0,     0],
       [  101,  2027,  3653, ...,     0,     0,     0],
       ...,
       [  101,  1996,  5896, ...,     0,     0,     0],
       [  101,  1037,  5667, ...,     0,     0,     0],
       [  101,  1037, 12090, ...,     0,     0,     0]])

In [24]:
np.array(padded_2).shape

(6920, 67)

**전체 문장에 대한 attention_mask_2 정의**

In [25]:
attention_mask_2 = np.where(padded_2 != 0, 1, 0)
attention_mask_2.shape

(6920, 67)

- model() 함수는 DistillBERT를 통해 문장을 실행합니다. 
- 처리 결과는 last_hidden_states_2로 반환됩니다.

In [26]:
input_ids_2 = torch.tensor(padded_2)  
attention_mask_2 = torch.tensor(attention_mask_2)

with torch.no_grad():
    last_hidden_states_2 = model(input_ids_2, attention_mask=attention_mask_2)

- 이후에 사용할 로지틱스 회귀 모델의 기능으로 last_hidden_states_2의 정보가 사용되므로 features_2 변수에 저장합니다.

In [27]:
features_2 = last_hidden_states_2[0][:,0,:].numpy()
features_2

array([[-0.21593435, -0.14028911,  0.00831076, ..., -0.13694832,
         0.5867005 ,  0.20112693],
       [-0.17262718, -0.14476153,  0.00223438, ..., -0.1744257 ,
         0.21386446,  0.37197465],
       [-0.05063373,  0.07203954, -0.02959727, ..., -0.0714895 ,
         0.7185238 ,  0.2622547 ],
       ...,
       [-0.06550944, -0.0518475 , -0.14094469, ..., -0.06450683,
         0.60223037,  0.21347877],
       [-0.08523139, -0.04869805, -0.08137526, ..., -0.13589348,
         0.39505622,  0.22889704],
       [-0.29436833, -0.09234679, -0.00831667, ..., -0.05159124,
         0.43497816,  0.28891575]], dtype=float32)

In [28]:
last_hidden_states_2

BaseModelOutput(last_hidden_state=tensor([[[-0.2159, -0.1403,  0.0083,  ..., -0.1369,  0.5867,  0.2011],
         [-0.2471,  0.2468,  0.1008,  ..., -0.1631,  0.9349, -0.0715],
         [ 0.0558,  0.3573,  0.4140,  ..., -0.2430,  0.1770, -0.5080],
         ...,
         [ 0.1864,  0.0193,  0.1864,  ..., -0.2175,  0.1604, -0.4050],
         [-0.1004,  0.0651,  0.1240,  ..., -0.1649,  0.3568,  0.1218],
         [-0.0114,  0.3297,  0.2317,  ..., -0.2362,  0.4217,  0.0895]],

        [[-0.1726, -0.1448,  0.0022,  ..., -0.1744,  0.2139,  0.3720],
         [ 0.0022,  0.1684,  0.1269,  ..., -0.1888, -0.0195, -0.0283],
         [ 0.0257, -0.2458,  0.0717,  ..., -0.4339,  0.1622,  0.0133],
         ...,
         [ 0.0466,  0.0850,  0.1801,  ..., -0.0279,  0.1878,  0.4022],
         [-0.2325,  0.0746,  0.1298,  ..., -0.1292,  0.0904,  0.3647],
         [-0.0655, -0.2214,  0.1827,  ..., -0.1624,  0.1421,  0.0963]],

        [[-0.0506,  0.0720, -0.0296,  ..., -0.0715,  0.7185,  0.2623],
         [ 

In [29]:
labels_2 = batch_2[1]
labels_2

0       1
1       0
2       0
3       1
4       1
       ..
6915    1
6916    0
6917    0
6918    0
6919    1
Name: 1, Length: 6920, dtype: int64

- 이제 데이터 세트를 훈련 세트와 테스트 세트로 분할하겠습니다(SST2 훈련 세트에서 6920개의 문장).

In [30]:
train_features_2, test_features_2, train_labels_2, test_labels_2 = train_test_split(features_2, labels_2)

In [31]:
lr_clf_2 = LogisticRegression()
lr_clf_2.fit(train_features_2, train_labels_2)

LogisticRegression()

**테스트 데이터 정확도 확인**

In [32]:
lr_clf_2.score(test_features_2, test_labels_2)

0.8572254335260115

## 모델 결과 비교

- 2000개의 데이터를 사용했을때 **81.8**의 score나왔는데, 전체 데이터를 다 사용하니 **85.02890173410405**의 score로 성능이 향상되었습니다.(모델 돌릴때마다 값이 조금 바뀌어서 처음 나온 결과로 작성하였습니다)

.

.

# BertForSequenceClassification


In [33]:
import gzip
import shutil
import time

import pandas as pd
import requests
import torch
import torch.nn.functional as F
import torchtext

import transformers
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification

In [34]:
import numpy as np
import pandas as pd
df = pd.read_csv('sst-train.tsv', delimiter='\t', header=None)

In [35]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NUM_EPOCHS = 3

In [36]:
df

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0


In [37]:
train_texts = df.iloc[:3900][0].values
train_labels = df.iloc[:3900][1].values

valid_texts = df.iloc[3900:5200][0].values
valid_labels = df.iloc[3900:5200][1].values

test_texts = df.iloc[5200:][0].values
test_labels = df.iloc[5200:][1].values

In [38]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [39]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [40]:
train_encodings[0]

Encoding(num_tokens=66, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [41]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [42]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

## Load Model

In [43]:
import torch

if torch.cuda.is_available():
    print("CUDA is available")
else:
    print("CUDA is not available")

CUDA is available


In [44]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train()

optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

## Train Model

In [45]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):

            ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs['loss'], outputs['logits']

            _, predicted_labels = torch.max(logits, 1)

            num_examples += labels.size(0)

            correct_pred += (predicted_labels == labels).sum()
    return correct_pred.float()/num_examples * 100

In [46]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_loader):
        
        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
        
        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 0001/0003 | Batch 0000/0244 | Loss: 0.7045
training accuracy: 95.59%
valid accuracy: 86.31%
Time elapsed: 0.45 min
Epoch: 0002/0003 | Batch 0000/0244 | Loss: 0.2252
training accuracy: 98.38%
valid accuracy: 84.62%
Time elapsed: 0.83 min
Epoch: 0003/0003 | Batch 0000/0244 | Loss: 0.2105
training accuracy: 99.49%
valid accuracy: 86.08%
Time elapsed: 1.22 min
Total Training Time: 1.22 min
Test accuracy: 87.67%


**BertForSequenceClassification로 모델링한 결과 3 에포크만에 87.67%의 Test accuracy를 보이며 Logistic Regression 방식보다 좋은 성능을 보였습니다.**

.

.

## Proper SST2 scores
참고로 이 데이터셋의 [최고 정확도 점수](http://nlpprogress.com/english/sentiment_analysis.html)는 현재 **96.8**입니다. DistilBERT는 이 작업에서 점수를 향상시키도록 훈련될 수 있습니다. 이 프로세스는 이 문장 분류 작업(다운스트림 작업이라고 부를 수 있음)에서 더 나은 성능을 달성할 수 있도록 BERT의 가중치를 업데이트하는 **Fine Tuning**이라는 프로세스입니다. 미세 조정된 DistilBERT는 **90.7**의 정확도 점수를 달성하는 것으로 나타났습니다. 전체 크기 BERT 모델은 **94.9**를 달성합니다.
